In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import libs.ngl as ngl
import datetime
from numba import njit
from numba.typed import Dict
from sklearn.metrics.pairwise import haversine_distances

In [57]:
mu = 1.0
beta = 0.1
x_min, x_max = -100,100
N = 10000
x = np.linspace(x_min, x_max, N)
dx = (x_max-x_min)/N
G = np.exp(-np.exp(-beta*x))
dt = 1e-4
for i in range(10000):
    mass = np.sum(G)
    G_x = np.diff(G, append=1.0)/dx
    G_xx = np.diff(G_x, append=0)/dx
    dG = dt*(0.5*G_xx + G*G-G - mu*np.exp(-beta*x)*G)
    G += dG
    delta_mass = -np.sum(G) + mass
    n_shift = int(np.ceil(delta_mass/dx))
    G = np.roll(G, n_shift)
    G[-n_shift:] = 1
    break

In [58]:
G

array([0.99995423, 0.99995432, 0.99995441, ..., 1.        , 1.        ,
       1.        ])

In [15]:
def load_hauksson():
    column_names = ["year","month","day", "hour", "minut", 'sec', "id",
                    "lat","lon", "depth", "mag", "nPhases", "azGap", "nearDist", "hErr", "vErr", "residual", "flag1", "flag2"]
    df = pd.read_csv("catalogs/sc_1981_2022q1_1d_3d_gc_soda.gc", sep="\s+", usecols=range(0,11), header=None,names=column_names[:11])
    df['time'] = pd.to_datetime(dict(year=df.year, month=df.month, day=df.day, hours=df.hour, minutes=df.minut, seconds=df.sec))
    return df

In [16]:
# grab GPS stations list
station_list = ngl.ngl_process_list(ngl.ngl_24h_2w) # daily measurements, with 2 weeks delay

In [17]:
# fill a dictionary with all the stations coordinates inside the region box
region_limits = [29.0, 37.0, -123.0, -112.0 ]
station_names, station_lats, station_lons  = ngl.get_all_stations_box(station_list, *region_limits)

In [18]:
df = load_hauksson()
df.head(5)

,year,month,day,hour,minut,sec,id,lat,lon,depth,mag,time
0,1981,1,1,1,49,29.357,3301561,33.73166,-118.81097,6.399,2.27,1981-01-01 01:49:29.357
1,1981,1,1,4,13,55.936,3301565,33.25355,-115.96716,6.347,2.26,1981-01-01 04:13:55.936
2,1981,1,1,5,20,14.634,3301566,34.18175,-117.30450,5.864,2.37,1981-01-01 05:20:14.634
3,1981,1,1,5,39,56.616,3301567,34.18031,-117.30303,5.991,1.60,1981-01-01 05:39:56.616
4,1981,1,1,8,23,18.236,3301570,34.00782,-117.16284,15.468,1.88,1981-01-01 08:23:18.236


In [19]:
# either download or load, depending whether they are on disk or not
rootpath = "csv_24/"
station_data = {}
for s_cnt, s_name in enumerate(station_names):
    df, status = ngl.ngl_retrieve_24h(rootpath, s_name)
    print(s_name, status)
    station_data[s_name] = df

59WE loaded
5PTS loaded
7ODM loaded
ACES loaded
ACSB loaded
ACSX loaded
AERO loaded
AGMT loaded
AGUA loaded
AIAH loaded
AISD loaded
ALPP loaded
ALTH loaded
ANA1 loaded
AOA1 loaded
APEX loaded
AQUA loaded
AR27 loaded
AR53 loaded
ARGU loaded
ARM1 loaded
ARM2 loaded
ASHM loaded
AUBR loaded
AVON loaded
AVRY loaded
AZAO loaded
AZBC loaded
AZBH loaded
AZBK loaded
AZDS loaded
AZFM loaded
AZGA loaded
AZGE loaded
AZGV loaded
AZKG loaded
AZLH loaded
AZLQ loaded
AZM1 loaded
AZMO loaded
AZMP loaded
AZNE loaded
AZPA loaded
AZPE loaded
AZPK loaded
AZPR loaded
AZPS loaded
AZPV loaded
AZPX loaded
AZQZ loaded
AZRY loaded
AZS1 loaded
AZSG loaded
AZTH loaded
AZU1 loaded
AZWA loaded
AZWB loaded
AZWE loaded
AZWK loaded
AZYC loaded
AZYU loaded
AZYW loaded
BADL loaded
BAK1 loaded
BAKR loaded
BAR1 loaded
BATM loaded
BBDM loaded
BBRY loaded
BCWR loaded
BEER loaded
BEMT loaded
BEPK loaded
BFLD loaded
BFSH loaded
BGIS loaded
BILL loaded
BIRD loaded
BKAP loaded
BKMS loaded
BKR1 loaded
BKR2 loaded
BLSA loaded
BLYT